# 학습 & 검증

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns


sns.set()

#-------------------- 차트 관련 속성 (한글처리, 그리드) -----------
plt.rcParams['font.family']= 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

#-------------------- 주피터 , 출력결과 넓이 늘리기 ---------------
# from IPython.core.display import display, HTML
from IPython.display import display, HTML

display(HTML("<style>.container{width:100% !important;}</style>"))
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', None)

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold, StratifiedKFold, cross_validate


from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.metrics import mean_squared_log_error

from sklearn.ensemble     import RandomForestRegressor
from sklearn.tree         import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.linear_model import LinearRegression, Ridge, Lasso
# ---- 추가 모델
from sklearn.ensemble     import AdaBoostRegressor, VotingRegressor
from xgboost              import XGBRegressor
from lightgbm             import LGBMRegressor



<table border=1 width=800>
<tr>
    <td>
    <img width='350' height='500' src="https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_006.png">
    <img width='350' height='500'  src="https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_007.png">
    </td>
</tr>
<tr>
    <td>
    <img width='350' height='500' src="https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_008.png">
    <img width='350' height='500'  src="https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_009.png">
    </td>
</tr>
<tr>
    <td>
    <img width='350' height='500' src="https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_010.png">
    <img width='350' height='500'  src="https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_011.png">
    </td>
</tr>
<tr>
    <td>
    <img width='350' height='500' src="https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_012.png">
    <img width='350' height='500'  src="https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_013.png">
    </td>
</tr>    
</table>

In [3]:
from sklearn.datasets import load_iris

In [4]:
dataset = load_iris()
dataset.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

* cross_val_score : 모델은 아님, 해당 모델 적용한 좋은 점수 보여주는 기능

* Grid Search Cv : RF 를 케바케로 많은 경우의 수를 갖고 있는 model's' --> 케바케를 가진 모델 만드는 친구, 학습시키는 친구 X
    - refit = 모델 O = 케바케 분석해서 핏 한 후, 해당 모델에 좋게 적용해줄게

상식)

    * process : 실행 중인 프로그램(multi process : 여러개를 동시에 실행 가능)
    * Thread : 프로세스 내에서 실제로 작업을 수행하는 주체

In [5]:
dataset['feature_names']

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [6]:
dataset['target']

#feature name 의 컬럼

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [7]:
df = pd.DataFrame(dataset['data'], columns = ['sl','sw','pl','pw'])
df

,sl,sw,pl,pw
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [8]:
df['target'] = dataset['target']
df.head()

,sl,sw,pl,pw,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   sl      150 non-null    float64
 1   sw      150 non-null    float64
 2   pl      150 non-null    float64
 3   pw      150 non-null    float64
 4   target  150 non-null    int32  
dtypes: float64(4), int32(1)
memory usage: 5.4 KB


In [10]:
df['target'].value_counts()

0    50
1    50
2    50
Name: target, dtype: int64

In [11]:
X = df.drop('target',axis = 1)
y = df['target']


X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 11, stratify=y) # stratify : y피쳐는 골고루 섞어주세요.

In [12]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((120, 4), (30, 4), (120,), (30,))

In [13]:
DT = DecisionTreeClassifier(random_state = 11)

DT.fit(X_train,y_train)
y_pred = DT.predict(X_test)
score = accuracy_score(y_test, y_pred)
score

1.0

In [14]:
y_train.value_counts(), y_test.value_counts()

(0    40
 2    40
 1    40
 Name: target, dtype: int64,
 1    10
 0    10
 2    10
 Name: target, dtype: int64)

# 교차검증(cross validation)

In [15]:
# 예시

In [16]:
age=20
gen=20

print('나이', 20 , "세" , "성별" , gen)
print(f"나이 {age} 성별 {gen}")

나이 20 세 성별 20
나이 20 성별 20


## KFold()

    * 일반적으로 회귀 모델에서 사용가능
    * test_score를 리스트로 return 한다.
<pre>

class sklearn.model_selection.KFold(n_splits=5, *, shuffle=False, random_state=None)

In [17]:
X = df.drop('target',axis = 1)
y = df['target']

In [18]:
kf = KFold(n_splits=3, shuffle=False, random_state=None)
for i, (train_index, test_index) in enumerate(kf.split(X)):
    print(f"Fold {i}:")
    # print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")
    print(df.loc[test_index, 'target'].value_counts())
    print('---'*30)

Fold 0:
  Test:  index=[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49]
0    50
Name: target, dtype: int64
------------------------------------------------------------------------------------------
Fold 1:
  Test:  index=[50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73
 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97
 98 99]
1    50
Name: target, dtype: int64
------------------------------------------------------------------------------------------
Fold 2:
  Test:  index=[100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117
 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]
2    50
Name: target, dtype: int64
------------------------------------------------------------------------------------------


In [19]:
X.loc[test_index]

,sl,sw,pl,pw
100,6.3,3.3,6.0,2.5
101,5.8,2.7,5.1,1.9
102,7.1,3.0,5.9,2.1
103,6.3,2.9,5.6,1.8
104,6.5,3.0,5.8,2.2
105,7.6,3.0,6.6,2.1
106,4.9,2.5,4.5,1.7
107,7.3,2.9,6.3,1.8
108,6.7,2.5,5.8,1.8
109,7.2,3.6,6.1,2.5


In [20]:
kf = KFold(n_splits=5, shuffle=True, random_state=11)

score_list=[]
for i, (train_index, test_index) in enumerate(kf.split(X)): #전체 데이터를 넣으면 알아서 등분 해주는 것
    print(f"Fold {i}:")
    # print(f"  Train: index={train_index}")
    # print(f"  Test:  index={test_index}")
    # print(df.loc[test_index, 'target'].value_counts())
    
    X_train = X.loc[train_index]
    X_test = X.loc[test_index]
    y_train = y.loc[train_index]
    y_test = y.loc[test_index]
    
    # X_test, y_test = X.loc[test_index], y.loc[test_index]
    # X_train, y_train= X.loc[train_index], y.loc[train_index]
    
    

#     print(f"  Train: index={train_index}")
#     print(f"  Test:  index={test_index}")
#     print(df.loc[test_index, 'target'].value_counts())
    
    
    DT = DecisionTreeClassifier(random_state = 11)
    DT.fit(X_train,y_train)
    y_pred = DT.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    
    score_list.append(score)

np.mean(score_list)
    
#     print(score)
    
#     print('---'*30)
    


Fold 0:
Fold 1:
Fold 2:
Fold 3:
Fold 4:


0.9466666666666667

## cross_validate()
* test_score 이외를 딕셔너리로 준다.
* 점수 : test_score, **train_score**
* 시간 : fit_time, score_time

In [21]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=11)
model = DecisionTreeClassifier(random_state=11)
score_list = cross_validate(model, X, y, scoring = 'accuracy', cv=kf, return_train_score=True)
print(score_list)

{'fit_time': array([0.00299168, 0.00299168, 0.00199389, 0.0039897 , 0.00199485]), 'score_time': array([0.00299215, 0.00199509, 0.00299311, 0.00099564, 0.00099707]), 'test_score': array([0.93333333, 0.96666667, 0.86666667, 0.96666667, 0.96666667]), 'train_score': array([1., 1., 1., 1., 1.])}


## cross_val_score
<pre>
<b>sklearn.model_selection.cross_val_score(estimator(모델), X, y=None, (데이터)
scoring(무슨 점수 내줄까?)=None, <fond color =blue> cv(몇 번 반복해줄까?=Fold)=None </fond>, fit_params(좋은 모델 찾으면 그거 알아서 적용해줄까?)=None</b>
groups=None, , pre_dispatch='2*n_jobs', n_jobs=None, verbose=0, error_score=nan)

In [22]:
model = DecisionTreeClassifier(random_state=11)
score_list = cross_val_score(model,X,y,scoring = 'accuracy',cv=5)
print(score_list)
print(np.mean(score_list))

#아래 K폴드 하고 list.append 한 것과 같은 방식이다. 

[0.96666667 0.96666667 0.9        0.96666667 1.        ]
0.9600000000000002


In [23]:
sf = StratifiedKFold(n_splits=5, shuffle=True, random_state=11)
score_list = cross_val_score(model,X,y,scoring = 'accuracy',cv=sf) #cv에 sf모델 자체를 넣는다.
print(score_list)
print(np.mean(score_list))

[0.93333333 0.96666667 0.86666667 0.96666667 0.96666667]
0.9400000000000001


## StratifiedKFold()


    * 일반적으로 분류 모델에서 사용 가능
    * 분류형인 골고루 데이터만 사용 가능
    * 
<pre>

class sklearn.model_selection.StratifiedKFold(n_splits=5, *, shuffle=False, random_state=None)

In [24]:
sf = StratifiedKFold(n_splits=3, shuffle=False, random_state=None) #0,1,2종 균등하게 섞어서 쪼개준다.
for i, (train_index, test_index) in enumerate(sf.split(X,y)): #y를 넣어줘야 비율을 알 수 있다.
    print(f"Fold {i}:")
    # print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")
    print(df.loc[test_index, 'target'].value_counts())
    print('---'*30)

Fold 0:
  Test:  index=[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115]
0    17
1    17
2    16
Name: target, dtype: int64
------------------------------------------------------------------------------------------
Fold 1:
  Test:  index=[ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132]
0    17
2    17
1    16
Name: target, dtype: int64
------------------------------------------------------------------------------------------
Fold 2:
  Test:  index=[ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  83  84
  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]
1    17
2    17
0    16
Name: targe

## GridSearchCV

<b>
    - S.K.Fold() + 모델 하이퍼파라미터 튜닝</b>

<pre>

class sklearn.model_selection.GridSearchCV(estimator, param_grid, *, scoring=None, n_jobs=None, refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', error_score=nan, return_train_score=False)

In [32]:
# class sklearn.tree.DecisionTreeClassifier
#(*, criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, 
#min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, class_weight=None, ccp_alpha=0.0)

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=11)

model = DecisionTreeClassifier(random_state=11)#사용할 모델

mydic = {'min_samples_split':[1,2,3],
        'min_samples_leaf':[1,2,3]} #파라미터 값 케바케로 지정. 파라미터값만큼 케이스가 생김 3*3 = 9개 모델 케이스

models = GridSearchCV(model, scoring = 'accuracy', cv=kf, param_grid = mydic) #9개 케이스의 모델들 생성

models.fit(X_train, y_train)

y_pred = models.predict(X_test)

score = accuracy_score(y_test, y_pred)


print(models.best_score_)

0.9666666666666666


In [26]:

#예측값과 스코어 models 로 바로 구하기.

In [27]:
models.best_params_

{'min_samples_leaf': 1, 'min_samples_split': 1}

In [28]:
models.best_estimator_

DecisionTreeClassifier(min_samples_split=1, random_state=11)

In [29]:
models

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=11, shuffle=True),
             estimator=DecisionTreeClassifier(random_state=11),
             param_grid={'min_samples_leaf': [1, 2, 3],
                         'min_samples_split': [1, 2, 3]},
             scoring='accuracy')

## K-Fold 외 교차검증(Cross Validation)
<pre>
K-Fold : K개의 폴드로 나누어 cv횟수만큼 반복 학습

홀드아웃 (Holdout)    : 비복원추출 
부트스트랩(Bootstrap) : 복원추출(63.2%)

LOOCV(Leave One Out Cross Validation)   : test 폴드(1개데이터)
LpOCV(Leave p     Out Cross Validation) : test 폴드(p개데이터)